# **Preprocessing Data**

**Mục tiêu: chuẩn bị dữ liệu sạch** để có thể đưa vào modeling:
   - Xử lý giá trị thiếu (missing values) cho cả numeric và categorical.
   - Nhận diện & xử lý outlier cho một số biến số quan trọng.
   - Chuẩn hóa / biến đổi thang đo (scaling) cho các cột numeric.
   - Mã hóa (encoding) các thuộc tính ordinal & categorical.

## Import thư viện & module trong src

In [1]:
import os
import sys
import numpy as np

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

from src.data_processing import (
    load_csv_as_str,
    summarize_missing,
    string_column_to_float,
    impute_categorical,
    ordinal_encode,
    one_hot_encode,
    clip_outliers_iqr,
    log_transform,
    zscore_standardize,
    EXPERIENCE_ORDER,
    COMPANY_SIZE_ORDER,
    LAST_NEW_JOB_ORDER,
)

RANDOM_STATE = 23120329
np.random.seed(RANDOM_STATE)

DATA_PATH = "../data/raw/aug_train.csv"

## Load dữ liệu

In [2]:
header, data_raw = load_csv_as_str(DATA_PATH, delimiter=",", has_header=True)

print("Số dòng   :", data_raw.shape[0])
print("Số cột    :", data_raw.shape[1])
print("Tên cột   :", header)

print("\n5 dòng đầu (raw, dạng string):")
for i in range(5):
    print(f"Row {i}:", data_raw[i])

Số dòng   : 19158
Số cột    : 14
Tên cột   : ['enrollee_id', 'city', 'city_development_index', 'gender', 'relevent_experience', 'enrolled_university', 'education_level', 'major_discipline', 'experience', 'company_size', 'company_type', 'last_new_job', 'training_hours', 'target']

5 dòng đầu (raw, dạng string):
Row 0: ['8949' 'city_103' '0.92' 'Male' 'Has relevent experience' 'no_enrollment'
 'Graduate' 'STEM' '>20' '' '' '1' '36' '1.0']
Row 1: ['29725' 'city_40' '0.7759999999999999' 'Male' 'No relevent experience'
 'no_enrollment' 'Graduate' 'STEM' '15' '50-99' 'Pvt Ltd' '>4' '47' '0.0']
Row 2: ['11561' 'city_21' '0.624' '' 'No relevent experience' 'Full time course'
 'Graduate' 'STEM' '5' '' '' 'never' '83' '0.0']
Row 3: ['33241' 'city_115' '0.789' '' 'No relevent experience' '' 'Graduate'
 'Business Degree' '<1' '' 'Pvt Ltd' 'never' '52' '1.0']
Row 4: ['666' 'city_162' '0.767' 'Male' 'Has relevent experience' 'no_enrollment'
 'Masters' 'STEM' '>20' '50-99' 'Funded Startup' '4' '8' '0

## Xử lí Missing value

#### **Review lại**

In [3]:
summarize_missing(data_raw, header=header)

Missing summary theo column:
enrollee_id               |      0 missing ( 0.00%)
city                      |      0 missing ( 0.00%)
city_development_index    |      0 missing ( 0.00%)
gender                    |   4508 missing (23.53%)
relevent_experience       |      0 missing ( 0.00%)
enrolled_university       |    386 missing ( 2.01%)
education_level           |    460 missing ( 2.40%)
major_discipline          |   2813 missing (14.68%)
experience                |     65 missing ( 0.34%)
company_size              |   5938 missing (30.99%)
company_type              |   6140 missing (32.05%)
last_new_job              |    423 missing ( 2.21%)
training_hours            |      0 missing ( 0.00%)
target                    |      0 missing ( 0.00%)


**Nhận xét:** Ta sẽ bỏ qua các feature có missing là 0.

#### **Chiến lược xử lý cho từng nhóm feature**

#### Numeric (`city_development_index`, `training_hours`)
- Không có missing nào. Bỏ qua phần Numeric.

#### Ordinal (`experience`, `company_size`, `last_new_job`)

- **Chiến lược:**
  - Missing → `"Unknown"` (coi là một trạng thái riêng).  
  - Dùng **ordinal encoding** theo thứ tự định nghĩa trước.  
  - Giá trị `"Unknown"` hoặc lạ → mã hóa thành **-1**.
- **Lý do:** Các biến này có **thứ tự tự nhiên** (ít → nhiều), nên cần giữ thông tin thứ bậc.  
  Đồng thời vẫn muốn mô hình thấy rõ trường hợp “không khai báo / không rõ” như một nhóm riêng.

#### Categorical (không có thứ tự)

Các biến như: `gender`, `enrolled_university`,
`education_level`, `major_discipline`, `company_type`,  ...

- **Chiến lược:** Missing → `"Unknown"`, sau đó **one-hot encoding** (có thêm cột `feature=Unknown`).
- **Lý do:** Các biến này chỉ là **nhãn phân loại**, không có thứ tự, nên one-hot là cách mã hóa tự nhiên nhất,  
  và giữ lại rõ ràng nhóm ứng viên “không cung cấp thông tin”. Ví dụ những người không điền `gender` thì có cùng một pattern chẳng hạn.


In [4]:
col_index = {name: i for i, name in enumerate(header)}

def get_col(name: str) -> np.ndarray:
    return data_raw[:, col_index[name]].astype(str)

n_rows = data_raw.shape[0]

city_dev_col       = get_col("city_development_index")
training_hours_col = get_col("training_hours")

experience_col      = get_col("experience")
company_size_col    = get_col("company_size")
last_new_job_col    = get_col("last_new_job")
gender_col          = get_col("gender")
enrolled_col        = get_col("enrolled_university")
education_level_col = get_col("education_level")
major_col           = get_col("major_discipline")
company_type_col    = get_col("company_type")
relexp_col          = get_col("relevent_experience")
city_col            = get_col("city")

experience_filled, _      = impute_categorical(experience_col, strategy="constant", constant_value="Unknown")
company_size_filled, _    = impute_categorical(company_size_col, strategy="constant", constant_value="Unknown")
last_new_job_filled, _    = impute_categorical(last_new_job_col, strategy="constant", constant_value="Unknown")

gender_filled, _          = impute_categorical(gender_col, strategy="constant", constant_value="Unknown")
enrolled_filled, _        = impute_categorical(enrolled_col, strategy="constant", constant_value="Unknown")
education_level_filled, _ = impute_categorical(education_level_col, strategy="constant", constant_value="Unknown")
major_filled, _           = impute_categorical(major_col, strategy="constant", constant_value="Unknown")
company_type_filled, _    = impute_categorical(company_type_col, strategy="constant", constant_value="Unknown")

print("Hoàn tất quá trình xử lí missing.")

Hoàn tất quá trình xử lí missing.


## Xử lý outlier & scale cho Numeric

1. **Chuyển chuỗi → số thực**  
   Dùng `string_column_to_float(...)` để chuyển toàn bộ cột từ `str` sang `float`,  
   giúp có thể tính toán thống kê, phát hiện outlier, chuẩn hóa,…

2. **Xử lý outlier bằng IQR clipping**  
   - Áp dụng `clip_outliers_iqr(city_values_raw)` và `clip_outliers_iqr(training_values_raw)`.  
   - Các giá trị quá nhỏ/quá lớn (nằm ngoài khoảng [Q1 − 1.5⋅IQR, Q3 + 1.5⋅IQR]) được kẹp lại về biên.  
   → Giảm ảnh hưởng của vài điểm cực trị nhưng vẫn giữ cấu trúc phân phối chung.

3. **Log-transform cho `training_hours`**  
   - Dữ liệu `training_hours` thường lệch phải (nhiều giá trị nhỏ, ít giá trị rất lớn).  
   - Dùng `log_transform(training_clip)` để nén các giá trị lớn, làm phân phối “tròn” hơn.  
   → Giúp mô hình ổn định hơn, giảm ảnh hưởng của một vài ứng viên training cực nhiều giờ.

4. **Chuẩn hóa Z-score cho cả hai biến**  
   - Áp dụng `zscore_standardize(city_clip)` và `zscore_standardize(training_log)`.  
   - Kết quả: mỗi biến có trung bình xấp xỉ 0 và độ lệch chuẩn xấp xỉ 1.  
   → Đưa các feature numeric về cùng thang đo, giúp thuật toán tối ưu hội tụ tốt hơn  
     và các hệ số (trong mô hình tuyến tính) dễ so sánh hơn về mặt độ lớn.

In [5]:
city_values_raw = string_column_to_float(city_dev_col)
training_values_raw = string_column_to_float(training_hours_col)

city_clip = clip_outliers_iqr(city_values_raw)
training_clip = clip_outliers_iqr(training_values_raw)

training_log = log_transform(training_clip)

city_z, city_mean, city_std = zscore_standardize(city_clip)
training_z, tr_mean, tr_std  = zscore_standardize(training_log)

print("city_development_index mean, std:", float(np.mean(city_z)), float(np.std(city_z)))
print("training_hours_log mean, std   :", float(np.mean(training_z)), float(np.std(training_z)))

city_development_index mean, std: -2.1270292251718208e-16 1.0
training_hours_log mean, std   : 3.9313879314422497e-16 1.0


## Ordinal encoding

Trong bước này, ta mã hoá 3 biến có thứ tự:

- `experience_filled`
- `company_size_filled`
- `last_new_job_filled`

Chiến lược:

1. Mỗi biến được gắn với một **thứ tự cố định** thông qua:
   - `EXPERIENCE_ORDER`
   - `COMPANY_SIZE_ORDER`
   - `LAST_NEW_JOB_ORDER`

2. Dùng `ordinal_encode(...)` để:
   - Các giá trị nằm trong danh sách thứ tự → mã hoá thành số nguyên 0, 1, 2, ... theo đúng vị trí trong list.
   - Các giá trị **không nằm trong list** (bao gồm `"Unknown"` đã fill ở bước missing) → mã hoá thành **-1** như một nhóm riêng.

3. Kết quả:
   - `exp_encoded`, `cs_encoded`, `lnj_encoded` là 3 vector số đã giữ được **thông tin thứ bậc** của từng biến.

In [6]:
exp_encoded, exp_map   = ordinal_encode(experience_filled, EXPERIENCE_ORDER)
cs_encoded, cs_map     = ordinal_encode(company_size_filled, COMPANY_SIZE_ORDER)
lnj_encoded, lnj_map   = ordinal_encode(last_new_job_filled, LAST_NEW_JOB_ORDER)

print("experience_encoded sample:", exp_encoded[:5])
print("company_size_encoded sample:", cs_encoded[:5])
print("last_new_job_encoded sample:", lnj_encoded[:5])

experience_encoded sample: [21. 15.  5.  0. 21.]
company_size_encoded sample: [-1.  2. -1. -1.  2.]
last_new_job_encoded sample: [1. 5. 0. 0. 4.]


## One-hot cho Categorical

In [7]:
cat_with_missing = [
    ("gender",              gender_filled),
    ("enrolled_university", enrolled_filled),
    ("education_level",     education_level_filled),
    ("major_discipline",    major_filled),
    ("company_type",        company_type_filled),
]

cat_no_missing = [
    ("relevent_experience", relexp_col),
    ("city",                city_col),
]

gender_oh, gender_cats                     = one_hot_encode(gender_filled)
enrolled_oh, enrolled_cats                 = one_hot_encode(enrolled_filled)
education_level_oh, education_level_cats   = one_hot_encode(education_level_filled)
major_oh, major_cats                       = one_hot_encode(major_filled)
company_type_oh, company_type_cats         = one_hot_encode(company_type_filled)

relexp_oh, relexp_cats                     = one_hot_encode(relexp_col)
city_oh, city_cats                         = one_hot_encode(city_col)

print("gender categories             :", len(gender_cats))
print("enrolled_university categories:", len(enrolled_cats))
print("education_level categories    :", len(education_level_cats))
print("major_discipline categories   :", len(major_cats))
print("company_type categories       :", len(company_type_cats))
print("relevent_experience categories:", len(relexp_cats))
print("city categories               :", len(city_cats))

gender categories             : 4
enrolled_university categories: 4
education_level categories    : 6
major_discipline categories   : 7
company_type categories       : 7
relevent_experience categories: 2
city categories               : 123


## Lưu kết quả 
#### Chuẩn bị

In [8]:
feature_blocks = []
feature_names  = []

feature_blocks.append(city_z.reshape(n_rows, 1))
feature_names.append("city_development_index_z")

feature_blocks.append(training_z.reshape(n_rows, 1))
feature_names.append("training_hours_log_z")

feature_blocks.append(exp_encoded.reshape(n_rows, 1))
feature_names.append("experience_ord")

feature_blocks.append(cs_encoded.reshape(n_rows, 1))
feature_names.append("company_size_ord")

feature_blocks.append(lnj_encoded.reshape(n_rows, 1))
feature_names.append("last_new_job_ord")

feature_blocks.append(gender_oh)
for c in gender_cats:
    feature_names.append(f"gender={c}")

feature_blocks.append(enrolled_oh)
for c in enrolled_cats:
    feature_names.append(f"enrolled_university={c}")

feature_blocks.append(education_level_oh)
for c in education_level_cats:
    feature_names.append(f"education_level={c}")

feature_blocks.append(major_oh)
for c in major_cats:
    feature_names.append(f"major_discipline={c}")

feature_blocks.append(company_type_oh)
for c in company_type_cats:
    feature_names.append(f"company_type={c}")

feature_blocks.append(relexp_oh)
for c in relexp_cats:
    feature_names.append(f"relevent_experience={c}")

feature_blocks.append(city_oh)
for c in city_cats:
    feature_names.append(f"city={c}")

X = np.hstack(feature_blocks)

target_col = data_raw[:, col_index["target"]]
y_num = target_col.astype(float)
y = (y_num == 1.0)

print("Shape X      :", X.shape)
print("Số feature   :", len(feature_names))
print("y shape      :", y.shape)

vals, cnts = np.unique(y, return_counts=True)
print("Phân phối y (False/True):", list(zip(vals, cnts)))

assert X.shape[1] == len(feature_names)
print("NaN trong X:", int(np.isnan(X).sum()))
print("Inf trong X:", int(np.isinf(X).sum()))

Shape X      : (19158, 158)
Số feature   : 158
y shape      : (19158,)
Phân phối y (False/True): [(np.False_, np.int64(14381)), (np.True_, np.int64(4777))]
NaN trong X: 0
Inf trong X: 0


### Lưu ra file

In [9]:
PROCESSED_DIR = "../data/processed"
os.makedirs(PROCESSED_DIR, exist_ok=True)

raw_filename = os.path.basename(DATA_PATH)
csv_out_path = os.path.join(PROCESSED_DIR, raw_filename)

data_out = np.column_stack([y.astype(int).reshape(-1, 1), X])
header_out = "target," + ",".join(feature_names)

np.savetxt(
    csv_out_path,
    data_out,
    delimiter=",",
    header=header_out,
    comments="",
    fmt="%.6f",
)

print("Đã lưu:", csv_out_path)

Đã lưu: ../data/processed\aug_train.csv
